Based on **Giancarlo Zaccone: Python Parallel Programming Cookbook, Chapter 3**

# Multiple processes
Traditionally, the way Python programmers have worked around the GIL and its 
effect on CPU-bound threads has been to **use multiple processes instead of multiple 
threads**. 

This approach (multiprocessing) has some **disadvantages**: 
having to launch multiple instances of the Python interpreter with all the 
**startup time and memory usage penalties** that this implies

The Python multiprocessing library, which is part of the standard library of the language, 
implements the shared memory programming paradigm, that is, the programming of a system 
that consists of one or more processors that have access to a common memory.

### How to spawn a process

The term "spawn" means the creation of a process by a parent process. 

The parent process can of course continue its execution asynchronously or wait until the child process ends its 
execution. 

The multiprocessing library of Python allows the spawning of a process through the following steps:

* Build the object process.
* Call its start() method. This method starts the process's activity.
* Call its join()method. It waits until the process has completed its work and exited

In [1]:
#Spawn a Process
import multiprocessing

def function(i):
    print ('called function in process: %s' %i)
    return

In [2]:

Process_jobs = []
for i in range(5):
    p = multiprocessing.Process(target=function, args=(i,))
    Process_jobs.append(p)
    p.start()
    p.join()



called function in process: 0
called function in process: 1
called function in process: 2
called function in process: 3
called function in process: 4


Without p.join(), the child process will sit idle and not be terminated, and then, you must 
manually kill it.